<a href="https://colab.research.google.com/github/JyYoungK/Stock-Price-Prediction/blob/main/Stock_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fbprophet
!pip install pip install yfinance
!pip install XlsxWriter

In [ ]:
import pandas_datareader.data as reader
import datetime as dt
import warnings
import numpy as np
import operator
from fbprophet import Prophet #AI
import yfinance as yf
import pandas as pd

warnings.simplefilter(action='ignore', category=FutureWarning)
end = dt.datetime.now() #Choose the End Day. Default: Today
ai_start = dt.datetime(end.year, end.month-1, end.day) #Choose the Start Day Default: 10years ago
vbs_start_1 = dt.datetime(end.year, end.month, end.day-20)
vbs_start_2 = dt.datetime(end.year, end.month, end.day-3)
hash = {}

#Put your stock ticker here----------------------------------------------------------------------------------------------------------------------------------------------
stocks = ['AAPL']
#Put your stock ticker here----------------------------------------------------------------------------------------------------------------------------------------------

gen_filtered_stocks = [] #All the stocks that are 90 below or 150 above and positive 2DC will be here
vbs_filtered_stocks = [] #All the stocks that are vbs filtered will be in here
ai_filtered_stocks = []  #All the stocks that are ai filtered will be in here

def general_filter():
    #Based on my beta testing stocks with less than 90 price and past 2 Daily change that are positive sells the most ---------------------------------------------------
    ticker2 = yf.Ticker(ticker)
    todays_data = ticker2.history(period='1d')
    if (len(todays_data) > 0):
        todays_end_price = float("%.2f" % float(todays_data['Close'][0]))
        past_data = ticker2.history(period='3d')
        end_price = float("%.2f" % float(past_data['Close'][0]))
        if (len(past_data) > 2):
            two_days_ago_price = float("%.2f" % float(past_data['Close'][2]))
            twoDC = float("%.2f" % float((two_days_ago_price-end_price)/end_price*100))
            if (twoDC>0 and (todays_end_price <= 90 or todays_end_price >= 150)):
                gen_filtered_stocks.append(ticker)
                print ("-", ticker, "has a today's price of", end_price, ", two daily change of", twoDC , "%")
            else:
                print ("-", ticker, "is not a good stock to buy today. Failed to pass the general analysis")
        else:
            print ("-", ticker, "is not a good stock to buy today. Failed to pass the general analysis")
    else:
        print ("-", ticker, "is not a good stock to buy today. Failed to pass the general analysis")

def get_ror(k):
    while True:
        try:
            df = reader.get_data_yahoo(ticker, vbs_start_1, end) #Choose your stock
            df['range'] = (df['High'] - df['Low']) * k
            df['target'] = df['Open'] + df['range'].shift(1) #Use yesterday's range

            # ror(net profit), np.where(If true = close/target, false = 1)
            df['ror'] = np.where(df['High'] > df['target'],
                                    df['Close'] / df['target'], 
                                    1)
                  
            ror = df['ror'].cumprod()[-2] #We only care about the 2nd last because that's the yesterday's price
            return ror
        except:
            print("-Error: Could not get a ror for", ticker, "Failed to pass the VBS analysis")
            pass
            
def get_target_price(k):
    #Filter the stocks using the best K value and applying the "Volatility Breakout Strategy" by Larry Williams. If Close price is greater than the Target price purchase it. 
    #1 ----------------------------------------------------------------------------------------------------------------------------------------------------------------------
    df = reader.get_data_yahoo(ticker, vbs_start_2, end) 
    df['range'] = (df['High'] - df['Low']) * float(k) # Find the range
    df['target'] = df['Open'] + df['range'].shift(1) #Find the target using the range

    if (len(df) > 2):
        close = float("%.2f" % float(df['Close'][2]))
        target= float("%.2f" % float(df['target'][2]))   
    else:
        close = 0
        target = 0  
        print ("-", ticker, "is not a good stock to buy today. Failed to pass the VBS analysis")   

    #1. Close price has to be greater than the target price.
    #2. Only purhcase stocks that are 90 or less 
    #3. Price change between 3days and 2days ago should be positive
    if (close>target):
      vbs_filtered_stocks.append(ticker)
      print ("-", ticker, "has a target price of", target)
    print ("-", ticker, "is not a good stock to buy today. Failed to pass the VBS analysis")   

def ai_filtering():
    while True:
        try:
            df = reader.get_data_yahoo(ticker, ai_start, end) #Choose your stock
            df = df.reset_index()
            if (len(df) > 22): #Checks if it has at least 20 days (because it doesn't include the weekends) worth of data
                df[['ds','y']] = df[['Date','Adj Close']]
                df_no_weekdays = df[df['ds'].dt.dayofweek < 5]
                model = Prophet(seasonality_mode='multiplicative', mcmc_samples=300)
                model.fit(df_no_weekdays)
                future = model.make_future_dataframe(periods=30) #Change the future prediction range here
                forecast = model.predict(future) #predicting formula. takes future parameter
                # model.plot(forecast) #Displays a graph
                #yhat = prediction, yhat_lower ~ yhat_upper = range
                # print ((forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]).tail(30)) #Next 30 days Prediction
                oneday_future = float("%.2f" % float(forecast['yhat'][len(forecast)-30]))
                tendays_future = float("%.2f" % float(forecast['yhat'][len(forecast)-20]))
                thirtydays_future = float("%.2f" % float(forecast['yhat'][len(forecast)-1]))
                if ((oneday_future < tendays_future) or (tendays_future < thirtydays_future)):
                    print ("-", ticker, "has a tomorrow's future price of", oneday_future, ", ten days future price of", tendays_future, "thirty days future price of", thirtydays_future)
                    ai_filtered_stocks.append(ticker)
                else:
                    print ("-", ticker, "is not a good stock to buy today. Failed to pass the AI analysis")
        except:
            print("-Error: Could not get an AI analysis for", ticker, ". Failed to pass the AI analysis")
            pass

for x, res in enumerate(stocks):
    ticker = res
    print ("Taking a general analysis on", ticker, "...", "(",x+1, "/", len(stocks), ") ", float("%.2f" % float((x+1)/len(stocks)*100)), "%")
    general_filter()

print ("These are the following stocks that passed the  general analysis:", gen_filtered_stocks)

Finding the best K value
for x, res in enumerate(gen_filtered_stocks): 
    ticker = res
    print ("Taking a VBS analysis on", ticker, "...", "(",x+1, "/", len(gen_filtered_stocks), ") ", float("%.2f" % float((x+1)/len(gen_filtered_stocks)*100)), "%")
    for k in np.arange(0.1, 1.0, 0.1):
        ror = get_ror(k)
        kvalue = "%.1f" % (k)
        rorvalue = "%f" % (ror)
        data = {kvalue:rorvalue}
        hash.update(data)

    highest_k_value = max(hash.items(), key=operator.itemgetter(1))[0]
    get_target_price(highest_k_value)
  
print ("These are the following stocks that passed the VBS analysis:", vbs_filtered_stocks)
#Doing an AI analysis  
for x, res in enumerate(vbs_filtered_stocks): 
    ticker = res
    print ("Taking an AI analysis on", ticker, "...", "(",x+1, "/", len(vbs_filtered_stocks), ") ", float("%.2f" % float((x+1)/len(vbs_filtered_stocks)*100)), "%")
    ai_filtering()

print ("These are the following stocks that passed the AI analysis:", ai_filtered_stocks)

# df = pd.DataFrame(ai_filtered_stocks)
# df.to_excel("full_analysis.xlsx")
